In [4]:
from xspec import *
m = Model("tbabs*bbody")
s = Spectrum("rxj1856_1k/spectrum_rxj1856_1k.pha")


Model TBabs<1>*bbody<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbody      kT         keV      3.00000      +/-  0.0          
   3    2   bbody      norm                1.00000      +/-  0.0          
________________________________________________________________________


1 spectrum  in use
 
Spectral Data File: rxj1856_1k/spectrum_rxj1856_1k.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.465e+01 +/- 1.210e-01
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-1024
  Telescope: eROSITA Instrument: TM1  Channel Type: PI
  Exposure Time: 1000 sec
 Using fit statistic: chi
 Using Response (RMF) File            /home/suro/SIXTE/installation/share/sixte/instruments/srg/erosita/erosita_pirmf_v20210719.rmf for Source 1
 Using Auxiliary Response (ARF) File  /home/suro/SIXTE/installation/share/sixte/instruments/srg/erosita/tm1_arf_fi

In [5]:
Xset.save(fileName='current')

In [ ]:
Xset.restore()